In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D  # 3Dプロット用

# ==============================================
# 【1. スクリプトと同じフォルダ内からCSVファイルを取得】
# ==============================================
script_dir = os.path.dirname(__file__)
csv_files = glob.glob(os.path.join(script_dir, "*.csv"))

if not csv_files:
    raise FileNotFoundError("同じフォルダにCSVファイルが見つかりません。")

# ==============================================
# 【2. 各物性値ごとのカラーバー表示範囲を設定】
# ==============================================
color_ranges = {
    "SiO2 Etching depth": (0, 1700),
    "SiO2 Etching rate": (0, 400),
    "Mask Etching depth": (0, 1400),
    "Mask Selectivity ratio": (1, 4),
    "Vpp": (0, 1000),
    "Vdc": (-800, 0)
}

# ==============================================
# 【3. 各CSVファイルごとに処理を実施】
# ==============================================
for file_path in csv_files:
    print(f"読み込み中: {file_path}")

    # Shift-JIS で読み込み
    df = pd.read_csv(file_path, encoding="shift_jis", header=None)

    # ==========================================
    # 【3-1. 物性値の名称（target_name）を取得】
    # 1行目から対象となる値の名前を抽出します。
    # ==========================================
    full_name = str(df.iloc[0,0])
    target_name = full_name.split("/", 1)[1].strip() if "/" in full_name else full_name.strip()
    print(f"対象値: {target_name}")

    # ==========================================
    # 【3-2. Coil, Platen, 測定値zを抽出】
    # X軸（Coil）、Y軸（Platen）、対応するZ（測定値）を取得します。
    # ==========================================
    coil_x = df.iloc[1,1:].astype(float).values
    platen_y = df.iloc[3:,0].astype(float).values
    z_values = df.iloc[3:,1:].astype(float).values

    # ==========================================
    # 【3-3. x, y, z のリスト展開】
    # CoilとPlatenのすべての組み合わせについて、測定値zを1次元配列化します。
    # ==========================================
    x_list, y_list, z_list = [], [], []
    for i, y_val in enumerate(platen_y):
        for j, x_val in enumerate(coil_x):
            x_list.append(x_val)
            y_list.append(y_val)
            z_list.append(z_values[i, j])
    x = np.array(x_list)
    y = np.array(y_list)
    z = np.array(z_list)

    # ==========================================
    # 【3-4. 2次の多項式回帰モデルの作成・学習】
    # Coil, Platenを説明変数として、測定値zを2次多項式でフィッティングします。
    # ==========================================
    X = np.column_stack((x, y))
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, z)

    # ==========================================
    # 【3-5. プロット用グリッドを作成し、回帰式で予測】
    # Coil・Platen値のグリッドを作成し、回帰モデルで予測値Ziを算出します。
    # ==========================================
    xi = np.linspace(0, 900, 100)
    yi = np.linspace(0, 250, 50)
    Xi, Yi = np.meshgrid(xi, yi)
    XY_poly = poly.transform(np.column_stack((Xi.ravel(), Yi.ravel())))
    Zi = model.predict(XY_poly).reshape(Xi.shape)

    # ==========================================
    # 【3-6. Vdc以外は下限値を0に制限（物理的に負にならないようクリップ）】
    # ==========================================
    if target_name != "Vdc":
        Zi = np.clip(Zi, 0, None)

    # ==========================================
    # 【3-7. カラーマップの範囲（vmin, vmax）設定】
    # ==========================================
    vmin, vmax = color_ranges.get(target_name, (None, None))

    # ==========================================
    # 【3-8. 3Dサーフェスと測定点のグラフ描画】
    # 回帰面（サーフェス）と実測値（赤い点）を3Dで可視化します。
    # ==========================================
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(Xi, Yi, Zi,
                           cmap=cm.viridis,
                           edgecolor='none',
                           alpha=0.9)
    ax.scatter(x, y, z, c='red', s=50, edgecolors="black", label="Measured")
    ax.set_xlabel("Coil [W]")
    ax.set_ylabel("Platen [W]")
    ax.set_zlabel(target_name)
    ax.set_title(f"3D Mapping of {target_name}")
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label=target_name)
    plt.legend()

    # ==========================================
    # 【3-9. 画像ファイルとして保存】
    # グラフ画像を物性値名ごとにPNG形式で保存します。
    # ==========================================
    safe_name = re.sub(r'[\\/:*?"<>|]', '_', str(target_name))
    output_file = f"{safe_name}_3D_surface.png"
    output_path = os.path.join(script_dir, output_file)
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()

    # ==========================================
    # 【3-10. 決定係数R²の表示（モデルの当てはまり指標）】
    # ==========================================
    r2 = model.score(X_poly, z)
    print(f"{target_name} の決定係数 R² = {r2:.3f}")
    print(f"3Dサーフェスのグラフを保存しました: {output_path}")